In [ ]:
# preparation database
import time
import sqlite3

dbpath = 'index.db'
path = 'cluster.db'
conn = sqlite3.connect(path)
cur = conn.cursor()

In [ ]:
def create_cluster_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Cluster (
                     addr INTEGER PRIMARY KEY,
                     cluster NOT NULL);
                ''')

def init_clusters():
    cur.execute('''INSERT INTO Cluster (addr, cluster)
                   SELECT id, -1 FROM other.TxID ORDER BY id ASC;''')

def attach_database():
    cur.execute('''ATTACH DATABASE ? AS other;''', (dbpath,))
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()

def synchronous(mode):
    sql = f'PRAGMA synchronous = {mode}'
    cur.execute(sql)
    conn.commit()

In [ ]:
create_cluster_table()

In [ ]:
try:
    synchronous('NORMAL')
    journal_mode('WAL')
    stime = time.time()
    attach_database()
    init_clusters()
    commit_transactions()
    etime = time.time()
    print(f'Copy Job done during {etime-stime}')
except KeyboardInterrupt:
    print(f'KeyboardInterrupt detected. Commit transactions.')
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
finally:
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
    conn.close()